In [2]:
%reset -f

In [3]:
# !pip install sentence-transformers
# !pip install tiktoken
# !pip install datasets
# !pip install faiss-gpu-cu12
# !pip install pdfplumber
# import torch
# import sentence_transformers
# import datasets
import pandas
import json

TRUE = True
FALSE = False

FRESH_START = FALSE

In [4]:
# [GET DATA TO BUILD RAG AROUND]
if FRESH_START:
  !mkdir -p ~/.kaggle
  !mv kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  !kaggle datasets download -d nandr39/bharatiya-nyaya-sanhita-dataset-bns
  !unzip bharatiya-nyaya-sanhita-dataset-bns.zip -d bns_data


In [5]:
### [REMOVE THE NOT AFTER THIS RUN/BEFORE NEXT RUN]

if FRESH_START:
  #!kaggle datasets download -d adarshsingh0903/legal-dataset-sc-judgments-india-19502024
  #!unzip legal-dataset-sc-judgments-india-19502024.zip -d sc_data
    # IT DOESN'T WORK T-T use downloaded version uploaded on drive instead
  !cp /content/drive/MyDrive/colab_stuff/court_hearings_rag/archive.zip /content/
  !unzip archive.zip -d /content/sc_judgments
  ### !cp -r /content/sc_judgments /content/drive/MyDrive/colab_stuff/court_hearings_rag/sc_judgments
else:
  pass
'''
TODO :: TODO, TODO, TODO TODO, TODO TODO, TODOOOOOOOOOOOOOOOOOOOOOO, TODO, TODO...
'''

'\nTODO :: TODO, TODO, TODO TODO, TODO TODO, TODOOOOOOOOOOOOOOOOOOOOOO, TODO, TODO...\n'

In [ ]:
"""
with pdfplumber.open("/content/sc_judgments/supreme_court_judgments/2020/Abhilasha_vs_Parkash_on_15_September_2020_1.PDF") as f:
  i = 0
  for page in f.pages:
    print(page.extract_text())
    print("\n\n")
    i += 1
    if i == 3:
      break
This confirms at on every page, no matter what year, the first line is the header and last the footer, we
henceforth remove them both. Though with time, minor details change like old ones have JUDGEMENT as heading
some new ones have J U D G E M E N T
"""

Abhilasha vs Parkash on 15 September, 2020
Abhilasha vs Parkash on 15 September, 2020
Equivalent citations: AIR 2020 SUPREME COURT 4355, AIRONLINE 2020 SC
727
Author: Ashok Bhushan
Bench: M.R. Shah, R. Subhash Reddy, Ashok Bhushan
REPORTABLE
IN THE SUPREME COURT OF INDIA
CRIMINAL APPELLATE JURISDICTION
CRIMINAL APPEAL NO. 615 of 2020
(arising out of SLP (Crl.) No.8260/2018)
ABHILASHA ...APPELLANT(S)
VERSUS
PARKASH & ORS. ...RESPONDENT(S)
J U D G M E N T
ASHOK BHUSHAN,J.
Leave granted.
2. This appeal has been filed by the appellant, daughter of respondent Nos. 1 and 2, challenging the
order of the High Court of Punjab and Haryana at Chandigarh dated 16.08.2018 by which order the
High Court dismissed the application under Section 482 Cr.P.C. filed by the appellant praying for
setting aside the order of the Judicial Magistrate First Class, Rewari dated 16.02.2011 as well as the
order dated 17.02.2014 passed by the Additional Sessions Judge, Rewari.
3. The brief facts necessary to be notic

In [6]:
def strip_headers_footers(pages):
    cleaned_pages = []

    for i, page in enumerate(pages):
        # i need to remove header and footer from all pages keeping all other data intact
        cleaned_pages.append(page)

    return "".join(cleaned_pages)

In [7]:
path_to_json = '/content/apex.jsonl' # advice gotten use json line, advice accepted
all_cases = []

def save_it(pages, file, file2, id):
  temp = {
      'id' : id,
      'source' : "Supreme Court",
      'title' : file2,
      'text' : strip_headers_footers(pages),
      'meta' : {
          'year' : file
          }
  }
  with open(path_to_json, "a", encoding='utf-8') as f:
    f.write(json.dumps(temp, ensure_ascii=False)) # dumps returns string of what dump was supposed to save
                                                  # we need to write line by line hence the way of f.write(json.dumps())
    f.write("\n") # this is how we make jsonl mate
all_cases.clear()

In [11]:
!cp /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl {path_to_json}


In [8]:
def _apex_remove_header_footer(text):

  HEADER = None
  FOOTER = f'Indian Kanoon - '
  HEADER, _ = current_case['text'].split('\n', 1) # in the text region the first line before '\n' gives us header, so only 1 split needed
  cleaned_current_case = []

  for line in current_case['text'].split('\n'):
    if line.startswith(HEADER) or line.startswith(FOOTER):
      continue
    else:
      cleaned_current_case.append(line)
    # a weird observation, even after loads() my footer of this and header of next page don't have \n in between
    # so like just detecting footers is enough to remove headers as well
  return '\n'.join(cleaned_current_case)  # we split on '\n' so we join on '\n\

In [38]:
# OBSERVATION 1 - if a sentance ends with \n but no fullstop before it, it was cut midway, remove \n
# OBSERVATION 2 - if a word has -\n it was cut midway, remove -\n
# OBSERVATION 3 - multiple -- serve some purpose, wait now, remove later
def _remove_enter_mid_words(text):
  formatted_sentance = []
  sentances = text.split('. ')
  for sentance in sentances:
    sentance = sentance.replace('-\n', '')
    sentance = sentance.replace('\n', ' ')
    formatted_sentance.append(sentance)
  return '.\n'.join(formatted_sentance)

In [44]:
'''SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE'''
# [OH BOY! WHAT HAVE I GOTTEN MYSELF INTO ?!]
import os
apex_raw = []

if FRESH_START:
  # import pdfplumber
  ROOT = "/content/sc_judgments/supreme_court_judgments"
  start_id = sum(1 for _ in open(path_to_json, "r", encoding="utf-8"))
  last_line = None

  print(f'starting fro i :: {start_id}')
  i = 1
  for file in sorted(os.listdir(ROOT)): # order of traversal is weird without sorted
    # print(file)
    print(f"\ncurrently in year :: {file}\n currently in case :: ", end=' ') # so i don't die waiting
    for file2 in sorted(os.listdir(f"{ROOT}/{file}")):
      if i < start_id:
        i += 1
        continue

      print(f"{i}", sep=' ', end=' ')
      try:
        with pdfplumber.open(f"{ROOT}/{file}/{file2}") as pdf:
          pages = []
          for page in pdf.pages:
            try:
              text = page.extract_text()
              if text:
                pages.append(text)
                # print(pages)
            except Exception:
              print(f'something wrong with page in pdf :: {i} ')
              with open('/content/errors.jsonl', 'a') as f:
                f.write(json.dumps(f' PAGE PROBLEM :: PDF :: {i} Year :: {file} Title :: {file2}'))
                f.write('\n')
                f.close()
          # only after pages has been appended with all data
          save_it(pages, file, file2, i)
          i += 1
      except Exception as e: # are these always meant to be used or always realized the need for
        print(f'something wrong with pdf :: {i}')
        with open('/content/errors.jsonl', 'a') as f:
          f.write(json.dumps(f' PDF PROBLEM :: PDF :: {i} Year :: {file} Title :: {file2}'))
          f.write('\n')
          f.close()
    !cp {path_to_json} /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl
    print(f'\n:: copied to drive ::\n')
else:
  with open(path_to_json, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
      # remove header and footer; we get header in first page first line
      # each line read here is a case in itself cuz jsonl
      current_case = json.loads(line)
      current_case['text'] = _remove_enter_mid_words(_apex_remove_header_footer(current_case['text']))
      apex_raw.append(current_case)
      if i == 100:
        print(f"broken at i = {i}")
        break
#print(apex_raw)
#print(len(apex_raw))
print(apex_raw[0]['text'][:7000])

broken at i = 100
A.K.
Gopalan vs The State Of Madras.Union Of India: ...
on 19 May, 1950 Equivalent citations: 1950 AIR 27, 1950 SCR 88, AIR 1950 SUPREME COURT 27, 1963 MADLW 638 Author: Hiralal J.
Kania Bench: Hiralal J.
Kania, Saiyid Fazal Ali, Mehr Chand Mahajan, B.K. Mukherjea PETITIONER: A.K.
GOPALAN Vs. RESPONDENT: THE STATE OF MADRAS.UNION OF INDIA: INTERVENER. DATE OF JUDGMENT: 19/05/1950 BENCH: KANIA, HIRALAL J.
(CJ) BENCH: KANIA, HIRALAL J.
(CJ) FAZAL ALI, SAIYID SASTRI, M.
PATANJALI MAHAJAN, MEHR CHAND DAS, SUDHI RANJAN MUKHERJEA, B.K. CITATION: 1950 AIR 27 1950 SCR 88 CITATOR INFO : F 1951 SC 157 (21) F 1951 SC 270 (5,6) F 1951 SC 301 (10) F 1951 SC 332 (344) E 1952 SC 75 (45) RF 1952 SC 123 (6) F 1952 SC 181 (6,27,29,33) D 1952 SC 196 (16) RF 1952 SC 252 (106) R 1952 SC 366 (16) E&F 1952 SC 369 (90,93) F 1953 SC 451 (7) E&F 1954 SC 92 (5,39) RF 1954 SC 119 (15) RF 1954 SC 728 (17) R 1955 SC 41 (6) F 1956 SC 108 (6) R 1957 SC 688 (6,8,9) R 1958 SC 163 (7,8,9,25,26,33,36,38

In [29]:
# i observe we can find very many more useful fields from this pdf. and hencewhy, i should parse
# all the documents and collect a list of all bold words, like CITATIONS JUDGEMENT etc and see
# the words that are present in all documents, that will give me a list of what further subdivisions
# are possible for each case
from collections import defaultdict
import string # to use string.punctuation which contains all filler used in english, useful, see
# print(string.punctuation)
all_cap_words = defaultdict(list)

for case in apex_raw:
  seen_in_case = defaultdict(int)
  for word in case["text"].split():
    w = word.strip(string.punctuation)
    if w.isupper() and len(w) > 1:
      seen_in_case[w] += 1
  for k,v in seen_in_case.items():
    all_cap_words[k].append(v)

for k,v in all_cap_words.items():
  if 50 < len(v) < 170:
    print(f"key :: {k} value :: {len(v)} values :: {v}")

'''
oh well lots of potential splits, but never occuring in ALL cases, so some/half the time, half the
columns/splits will go empty. Can do it in future, consider the columns where the count of word does
not change as well, like JUDGEMENT occurs two times in all cases, can be 3 its fine, but something like ALI
or SC occurs very variably 12 3 1 ... so i will drop them to get certain names that can be used as
subdivisions of text column, then will have to manually see how they are written to embed them as well
TODO
'''

key :: AIR value :: 96 values :: [3, 3, 3, 1, 1, 3, 3, 2, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 2, 1, 2, 2, 1, 3, 2, 3, 3, 1, 1, 3, 3, 2, 1, 3, 3, 1, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 3, 1, 3, 3, 2, 3, 2, 3, 1, 2, 1, 3, 3, 2, 3, 2, 3, 1, 2, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 1, 2, 2, 1, 1, 2, 1, 2, 3, 3, 3, 3]
key :: SCR value :: 72 values :: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
key :: SUPREME value :: 77 values :: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
key :: COURT value :: 77 values :: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Mate the time estimate is more than 10 hours, i HAVE to use multiprocessing here mate.

Also dam this text stuff useful to make notes instead of writing all that in comments. Definately underestimated notebook all this time.

Colab gives 1 worker, whats' the difference to using single threaded code then?!

In [ ]:
'''
MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE

Well so the idea is to make worker/threads and thread will go parse pdf for us, and we will just collect
data from our threads and write them orderly in jsonl to prevent conflicts, cheap way. Hence this
block is responsible for going to fetch the pdf, read it, and return us one line of our jsonl
'''
def process_pdf(args): # i can pass only 1 argument hence a tuple/struct
  year, filename, pdf_path, id = args
  pages = []

  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      text = page.extract_text()
      if text:
        pages.append(text)

  return {
    "id": id,
    "source": "Supreme Court",
    "title": filename,
    "text": strip_headers_footers(pages),
    "meta": {"year": year}
    }


In [ ]:
'''
MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE

Just like before we again parse all our folders and subfolders to read pdf(s)
But instead of reading directly we are making a list of 'tasks' we need to do
ie pfd(s) we need to read which contains, ofc, most importantly, the path to pdf
and some metadata that needs be stored in jsonl/datastructure of ours.
'''
import pdfplumber
import os
ROOT = "/content/sc_judgments/supreme_court_judgments"

tasks = [] # contains all files, weirdly enough i now know how many files i need to process
i = 1

for year in sorted(os.listdir(ROOT)):
  for file2 in sorted(os.listdir(f"{ROOT}/{year}")):
    tasks.append((
      year,
      file2,
      f"{ROOT}/{year}/{file2}",
      i
    ))
    i += 1


In [ ]:
"""
MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE

"""
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import json

workers = max(1, multiprocessing.cpu_count() - 1)
print(f"the number of workers we have is :: {workers}")

total = len(tasks)
done = 0

# this (main process) is responsible for writing to pdf, so we open pdf
with open(path_to_json, "a", encoding="utf-8") as f:
  # and allocate our minions/threads their task they need to do
  with ProcessPoolExecutor(max_workers=workers) as executor:
    # and for each worker, we write back the result
    for result in executor.map(process_pdf, tasks):
      done += 1
      print(f"\rProcessed {done}/{total} PDFs :: year - {tasks[done-1][0]}")
      if result is not None:
        f.write(json.dumps(result, ensure_ascii=False))
        f.write("\n")
      if result % 100 == 0:
        !cp {path_to_json} /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl


the number of workers we have is :: 1
Processed 1/26688 PDFs :: year - 1950
Processed 2/26688 PDFs :: year - 1950
Processed 3/26688 PDFs :: year - 1950
Processed 4/26688 PDFs :: year - 1950
Processed 5/26688 PDFs :: year - 1950
Processed 6/26688 PDFs :: year - 1950
Processed 7/26688 PDFs :: year - 1950
Processed 8/26688 PDFs :: year - 1950
Processed 9/26688 PDFs :: year - 1950
Processed 10/26688 PDFs :: year - 1950
Processed 11/26688 PDFs :: year - 1950
Processed 12/26688 PDFs :: year - 1950
Processed 13/26688 PDFs :: year - 1950
Processed 14/26688 PDFs :: year - 1950
Processed 15/26688 PDFs :: year - 1950
Processed 16/26688 PDFs :: year - 1950
Processed 17/26688 PDFs :: year - 1950
Processed 18/26688 PDFs :: year - 1950
Processed 19/26688 PDFs :: year - 1950
Processed 20/26688 PDFs :: year - 1950
Processed 21/26688 PDFs :: year - 1950
Processed 22/26688 PDFs :: year - 1950
Processed 23/26688 PDFs :: year - 1950
Processed 24/26688 PDFs :: year - 1950
Processed 25/26688 PDFs :: year - 1

KeyboardInterrupt: 

This is actually cooler than multiprocessing cuz, ProcessPoolExecuter() actually gives us FULL FELEGED PROCESSES that are python process- each with own memory cputime and shit, and executor is responsible for handling it

In [ ]:
dataset_supereme_high_court = datasets.load_dataset("opennyaiorg/InJudgements_dataset", token="")


README.md:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

data/train-00000-of-00002-add4caaf8fbc6a(…):   0%|          | 0.00/150M [00:00<?, ?B/s]

data/train-00001-of-00002-09ac6bd45d6b36(…):   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11970 [00:00<?, ? examples/s]

In [ ]:
# [BEAUTIFY BNS DATASET]
SAVE_PATH1 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns.json"

if FRESH_START:
  bns_raw = pandas.read_csv("/content/bns_data/bns_sections.csv")
  print(bns_raw.head())
  bns = []
  for i, entries in bns_raw.iterrows():
    temp = {
        'id' : i+1,
        'chapter': entries['Chapter'],
        'section': entries['Section'],
        'title': entries['Section _name'],
        'text': entries['Description']
    }
    bns.append(temp)
  print(bns[0])

  with open(SAVE_PATH1, 'w') as f:
    json.dump(bns, f)
else:
  with open(SAVE_PATH1, 'r') as f:
    bns = json.load(f)

   Chapter    Chapter_name Chapter_subtype  Section  \
0        1     Preliminary     Preliminary        1   
1        1     Preliminary     Preliminary        2   
2        1     Preliminary     Preliminary        3   
3        2  Of punishments  Of punishments        4   
4        2  Of punishments  Of punishments        5   

                               Section _name  \
0  Short title, commencement and application   
1                               Definitions.   
2                       General explanations   
3                                Punishments   
4                   Commutation of sentence    

                                         Description  
0  (1) This Act may be called the Bharatiya Nyaya...  
1  In this Sanhita, unless the context otherwise ...  
2  (1) Throughout this Sanhita every definition o...  
3  The punishments to which offenders are liable ...  
4  The appropriate Government may, without the co...  
{'id': 1, 'chapter': 1, 'section': 1, 'title': 'Sh

In [ ]:
# [BEAUTIFY SUP HI COURT DATASET]
SAVE_PATH2 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_courts.json"

# [THE TEXT DATA IS TOO NOISY WITH HTML TAGS]
def clean_html(text):
  import re
  text = re.sub(r"<span[^>]*>.*?</span>", " ", text, flags=re.DOTALL) # REMOVE 'EM HTML
  text = re.sub(r"<[^>]+>", " ", text) # REMOVE 'EM HTML
  text = re.sub(r"[=\-_.]{4,}", " ", text)

  return text

# print(dataset_supereme_high_court)
# print(dataset_supereme_high_court['train'][0])
if FRESH_START:
  high_courts_raw = dataset_supereme_high_court['train']
  high_courts_raw[0]
  print(high_courts_raw[0].keys())
  high_courts = []
  for i, cases in enumerate(high_courts_raw):
    temp = {
        'id' : i+1,
        'court_details' : f"{cases['Court_Type']} | {cases['Case_Type']} | {cases['Court_Name']}",
        'cited_by' : cases['Cited_by'],
        'title': cases['Titles'],
        # 'text' : " ".join(cases["Text"].split()) # too mych filler mate NEXT DAY REALIZATION : without spaces i have no idea where
        # a paragraph or context ends, so i end up making random chunks distributing meaning(s) very badly, keep it for now, remove
        # at chunking step instead
        'text' : clean_html(cases["Text"])
    }
    high_courts.append(temp)

  print(high_courts[0])
  with open(SAVE_PATH2, "w") as f:
    json.dump(high_courts, f)
else:
  with open(SAVE_PATH2, "r") as f:
    high_courts = json.load(f)

dict_keys(['Titles', 'Court_Name', 'Cites', 'Cited_by', 'Doc_url', 'Text', 'Doc_size', 'Case_Type', 'Court_Type', 'Court_Name_Normalized'])
{'id': 1, 'court_details': 'High_Court | Land&Property | Patna High Court - Orders', 'cited_by': 9, 'title': 'Maheshwar Mandal & Anr vs The State Of Bihar & Ors on 24 June, 2014', 'text': 'IN THE HIGH COURT OF JUDICATURE AT PATNA\n                  Civil Writ Jurisdiction Case No.1091 of 2013\n                                        With\n                      Interlocutory Application No. 1572 of 2013\n                                         In\n                  Civil Writ Jurisdiction Case No. 1091 of 2013\n \n1. Maheshwar Mandal S/O Late Bachchi Mandal\n2. Sanjay Mandal S/O Maheshwar Mandal, both resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivision- Forbesganj, District Araria     Petitioners Versus\n1. The State of Bihar\n2. Divisional Commissioner, Purnea\n3. Land Reforms Deputy Collector, Forbesganj, Araria     Resp

In [ ]:
"""
[FURTHER PRETTIFYING OF DATA]
well i see no point in doing this, but apprearently it helps later on figuring what went wrong in
RAG design and extending it in future as well.
"""

new_bns = []
for entries in bns:
  temp = {
      'id' : f"BNS_{entries['id']}",
      'source' : "BNS",
      'title' : entries['title'],
      'text' : entries['text'],
      'meta' : {
          'chapter' : entries['chapter'],
          'section' : entries['section']
          }
  }
  new_bns.append(temp)
bns = new_bns
del new_bns

new_court = []
for entries in high_courts:
  temp = {
      'id' : f"COURTS_{entries['id']}",
      'source' : "Courts",
      'title' : entries['title'],
      'text' : entries['text'],
      'meta' : {
          'cited_by' : entries['cited_by'],
          'court_details' : entries['court_details']
          }
  }
  new_court.append(temp)
court = new_court # yeh i changed name.
del new_court
del high_courts
print(court[0])
print("\n\n")
print(bns[0])

In [ ]:
# well now since i hesitantly unified representation of both(infuture three:: TODO TODO TODO TODO TODOOOOOOOOOOOOO) datasets' embeddings
# i can write a single function to break 'em all up in chunks and save the relevant info only.

def make_chunks(text, tokenizer, token_len=400, overlap_len=60):
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    chunks = []
    current_tokens = []

    for para in paragraphs:
        para_tokens = tokenizer.encode(para)

        # if paragraph alone is too big, split it
        if len(para_tokens) > token_len:
            for i in range(0, len(para_tokens), token_len - overlap_len):
                chunks.append(para_tokens[i:i + token_len])
            current_tokens = []
            continue

        # if adding paragraph exceeds budget → flush
        if len(current_tokens) + len(para_tokens) > token_len:
            chunks.append(current_tokens)
            # overlap
            current_tokens = current_tokens[-overlap_len:] if overlap_len else []

        current_tokens.extend(para_tokens)

    if current_tokens:
        chunks.append(current_tokens)

    return chunks


In [47]:
'''
[well chunking it is mate]
assumed that my input will be a data structure which will contain a subfield "text" that i need
to chunk. And paragraphs will be seperated by \n so those are my assumptions on input
'''
import numpy as np
from sentence_transformers import SentenceTransformer

def _label_chunks(chunks, data):
  chunks_final = []
  for i, chunk in enumerate(chunks):
    temp = {
      'chunk_id' : f"{data['source']}_{data['id']}_{i}",
      'chunk_text' : chunk,
      'parent_id' : data['id'],
      'source' : data['source']
          # this should be enough to retireve original data if needed
    }
    chunks_final.append(temp)
  return chunks_final

def make_chunks(data, tokenizer, para_overlap=0, max_tokens=5000, token_overlap=650):
  # split & clean paragraphs
  paragraphs = [p.strip() for p in data["text"].split("\n") if p.strip()]

  chunks = []
  chunk = []
  running_len = 0

  # precompute token lengths once
  para_tokens = [len(tokenizer.encode(p)) for p in paragraphs]
  print(f'precomputed tokens :: ')

  for i, paragraph in enumerate(paragraphs):
    p_len = para_tokens[i]

    # handle absurdly long single paragraph
    if p_len > max_tokens:
      continue  # or sentence-split later

    # if adding paragraph would overflow
    if running_len + p_len > max_tokens:
      # save current chunk
      if chunk:
          chunks.append(chunk.copy())

      # start new chunk with overlap
      chunk = []
      running_len = 0

      if para_overlap > 0:
        for j in range(max(0, i - para_overlap), i):
          overlap_len = para_tokens[j]
          if running_len + overlap_len > token_overlap:
            break
          chunk.append(paragraphs[j])
          running_len += overlap_len

    # add current paragraph
    chunk.append(paragraph)
    running_len += p_len

  # save final chunk
  if chunk:
    chunks.append(chunk.copy())

  return _label_chunks(chunks, data)
chunks = make_chunks(apex_raw[0], SentenceTransformer('all-MiniLM-L6-v2').tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (900 > 256). Running this sequence through the model will result in indexing errors


precomputed tokens :: 


In [ ]:
'''
hear the news, faiss returns integer index of what was used to embed, so, embedding_list[i]  <same order>  metadata_list[i]
'''


In [ ]:
'''
make embedding and select model
'''
model_name = 'all-MiniLM-L6-v2'
# remember that faiss is cpu first, need to move everything to gpu manually

def embed(model_name, embedding_name, chunks):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = SentenceTransformer(model_name, device=device)
  path = f'/content/embedding_{embedding_name}.npy'

  if FRESH_START:
    # model.to(device) not adviced, do at creation, else things break
    chunk_texts = [chunk['chunk_text'] for chunk in chunks]
    embeddings = model.encode(
      chunk_texts,
      batch_size = 64,
      convert_to_numpy=True, # remember faiss numpy
      show_progress_bar=True,
      normalize_embeddings=True,
      device=device
      )
    np.save(path, embeddings)
  else:
    embeddings = np.load(path)
  return embeddings

embed(model_name, 'apex', chunks)